In [ ]:
from pyspark.sql import SparkSession 
from pyspark.sql.types import StructType, StructField, IntegerType, StringType

# stop any existing Spark session, if Spark is already running, creating a new session might fail ***
try:
    spark.stop()
except Exception:
    pass

# create session with adjusted memory settings based on your cluster
# .config("spark.local.dir", r"E:\Apache Spark\spark-temp"): change the spark local dir, as the c disk memory is not enough, may cause Py4JJavaError exception ***
spark = SparkSession.builder.appName("BookRecommendCSV") \
    .config("spark.executor.memory", "16g") \
    .config("spark.driver.memory", "8g") \
    .config("spark.local.dir", r"E:\Apache Spark\spark-temp") \
    .getOrCreate() 

In [298]:
import pyspark
print(pyspark.__file__)

c:\Users\MECHREVO\AppData\Local\Programs\Python\Python312\Lib\site-packages\pyspark\__init__.py


In [299]:
spark.conf.get("spark.executor.memory")
spark.conf.get("spark.driver.memory")


'8g'

In [300]:
spark


In [301]:
# Read the CSV file with specified options
books = spark.read \
    .format("csv") \
    .option("sep", ";") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .option("mode", "DROPMALFORMED") \
    .load("data/BX-Books.csv")

In [302]:
books.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [303]:
books.columns

['ISBN',
 'Book-Title',
 'Book-Author',
 'Year-Of-Publication',
 'Publisher',
 'Image-URL-S',
 'Image-URL-M',
 'Image-URL-L']

In [304]:
books.show()

+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|      ISBN|          Book-Title|         Book-Author|Year-Of-Publication|           Publisher|         Image-URL-S|         Image-URL-M|         Image-URL-L|
+----------+--------------------+--------------------+-------------------+--------------------+--------------------+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|               2002|Oxford University...|http://images.ama...|http://images.ama...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|               2001|HarperFlamingo Ca...|http://images.ama...|http://images.ama...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|               1991|     HarperPerennial|http://images.ama...|http://images.ama...|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari

In [305]:
rename_dict = {
    "Book-Title": "title",
    "Book-Author": "author",
    "Year-Of-Publication": "year",
    "Publisher": "publisher",
    "Image-URL-L": "image_url"
}

for old_name, new_name in rename_dict.items():
    books = books.withColumnRenamed(old_name, new_name)

In [306]:
books = books.select('ISBN', 'title', 'author', 'year', 'publisher', 'image_url')

In [307]:
books.show()

+----------+--------------------+--------------------+----+--------------------+--------------------+
|      ISBN|               title|              author|year|           publisher|           image_url|
+----------+--------------------+--------------------+----+--------------------+--------------------+
|0195153448| Classical Mythology|  Mark P. O. Morford|2002|Oxford University...|http://images.ama...|
|0002005018|        Clara Callan|Richard Bruce Wright|2001|HarperFlamingo Ca...|http://images.ama...|
|0060973129|Decision in Normandy|        Carlo D'Este|1991|     HarperPerennial|http://images.ama...|
|0374157065|Flu: The Story of...|    Gina Bari Kolata|1999|Farrar Straus Giroux|http://images.ama...|
|0393045218|The Mummies of Ur...|     E. J. W. Barber|1999|W. W. Norton &amp...|http://images.ama...|
|0399135782|The Kitchen God's...|             Amy Tan|1991|    Putnam Pub Group|http://images.ama...|
|0425176428|What If?: The Wor...|       Robert Cowley|2000|Berkley Publishin...|ht

In [342]:
num_rows = books.count()
num_cols = len(books.columns)
print(num_rows, num_cols)

271379 6


In [ ]:
# read the User csv file
users = spark.read \
    .format("csv") \
    .option("sep", ";") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .option("mode", "DROPMALFORMED") \
    .load("data/BX-Users.csv")

In [310]:
users.show()

+-------+--------------------+----+
|User-ID|            Location| Age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
|      3|moscow, yukon ter...|NULL|
|      4|porto, v.n.gaia, ...|  17|
|      5|farnborough, hant...|NULL|
|      6|santa monica, cal...|  61|
|      7| washington, dc, usa|NULL|
|      8|timmins, ontario,...|NULL|
|      9|germantown, tenne...|NULL|
|     10|albacete, wiscons...|  26|
|     11|melbourne, victor...|  14|
|     12|fort bragg, calif...|NULL|
|     13|barcelona, barcel...|  26|
|     14|mediapolis, iowa,...|NULL|
|     15|calgary, alberta,...|NULL|
|     16|albuquerque, new ...|NULL|
|     17|chesapeake, virgi...|NULL|
|     18|rio de janeiro, r...|  25|
|     19|           weston, ,|  14|
|     20|langhorne, pennsy...|  19|
+-------+--------------------+----+
only showing top 20 rows



In [311]:
rename_dict = {
    "User-ID": "user_id",
    "Location": "location",
    "Age": "age",
}


for old_name, new_name in rename_dict.items():
    users = users.withColumnRenamed(old_name, new_name)

In [312]:
users.show(2)

+-------+--------------------+----+
|user_id|            location| age|
+-------+--------------------+----+
|      1|  nyc, new york, usa|NULL|
|      2|stockton, califor...|  18|
+-------+--------------------+----+
only showing top 2 rows



In [313]:
# Read the Ratings CSV file
ratings = spark.read \
    .format("csv") \
    .option("sep", ";") \
    .option("header", "true") \
    .option("encoding", "latin1") \
    .option("mode", "DROPMALFORMED") \
    .load("data/BX-Book-Ratings.csv")

In [314]:
ratings.show(5)

+-------+----------+-----------+
|User-ID|      ISBN|Book-Rating|
+-------+----------+-----------+
| 276725|034545104X|          0|
| 276726|0155061224|          5|
| 276727|0446520802|          0|
| 276729|052165615X|          3|
| 276729|0521795028|          6|
+-------+----------+-----------+
only showing top 5 rows



In [343]:
num_rows = ratings.count()
num_cols = len(ratings.columns)
print(num_rows, num_cols)  # the same as ratings.shape

59850 9


In [316]:
rename_dict = {
    "User-ID": "user_id",
    "Book-Rating": "rating"
}


for old_name, new_name in rename_dict.items():
    ratings = ratings.withColumnRenamed(old_name, new_name)

In [317]:
ratings.show(5)

+-------+----------+------+
|user_id|      ISBN|rating|
+-------+----------+------+
| 276725|034545104X|     0|
| 276726|0155061224|     5|
| 276727|0446520802|     0|
| 276729|052165615X|     3|
| 276729|0521795028|     6|
+-------+----------+------+
only showing top 5 rows



In [318]:
from pyspark.sql.functions import col

ratings.groupBy("user_id") \
       .count() \
       .withColumnRenamed("count", "rating_count") \
       .orderBy(col("rating_count"), ascending=False) \
       .show()


+-------+------------+
|user_id|rating_count|
+-------+------------+
|  11676|       13602|
| 198711|        7550|
| 153662|        6109|
|  98391|        5891|
|  35859|        5850|
| 212898|        4785|
| 278418|        4533|
|  76352|        3367|
| 110973|        3100|
| 235105|        3067|
| 230522|        2991|
|  16795|        2948|
| 234623|        2674|
|  36836|        2529|
|  52584|        2512|
| 245963|        2507|
| 204864|        2504|
|  55492|        2459|
| 185233|        2448|
| 171118|        2421|
+-------+------------+
only showing top 20 rows



In [319]:
ratings.groupBy("user_id").count().orderBy("count", ascending=False).show() # get every user's ratings for the books, sorted in descending order

+-------+-----+
|user_id|count|
+-------+-----+
|  11676|13602|
| 198711| 7550|
| 153662| 6109|
|  98391| 5891|
|  35859| 5850|
| 212898| 4785|
| 278418| 4533|
|  76352| 3367|
| 110973| 3100|
| 235105| 3067|
| 230522| 2991|
|  16795| 2948|
| 234623| 2674|
|  36836| 2529|
|  52584| 2512|
| 245963| 2507|
| 204864| 2504|
|  55492| 2459|
| 185233| 2448|
| 171118| 2421|
+-------+-----+
only showing top 20 rows



In [320]:
ratings_counts = ratings.groupBy("user_id").count()

In [321]:
ratings_counts = ratings_counts.filter(ratings_counts["count"] > 200)

In [345]:
ratings_counts.show(5)

+-------+-----+
|user_id|count|
+-------+-----+
|  32773|  745|
|  31391|  208|
|  26593|  272|
|  28360|  247|
|  19085|  240|
+-------+-----+
only showing top 5 rows



In [323]:
user_ids = ratings_counts.select("user_id")

user_id_list = [row['user_id'] for row in user_ids.collect()]

In [324]:
user_id_list

['32773',
 '31391',
 '26593',
 '28360',
 '19085',
 '29855',
 '28591',
 '31826',
 '16966',
 '23872',
 '30511',
 '8681',
 '26544',
 '25981',
 '10447',
 '25601',
 '23902',
 '13552',
 '30509',
 '2977',
 '6251',
 '21576',
 '7915',
 '8936',
 '31846',
 '18067',
 '19664',
 '11601',
 '14422',
 '30711',
 '2766',
 '35836',
 '16916',
 '11993',
 '32721',
 '8245',
 '6575',
 '8890',
 '16106',
 '16795',
 '21252',
 '24194',
 '13850',
 '32195',
 '30533',
 '3757',
 '15408',
 '20201',
 '35050',
 '10819',
 '35433',
 '23768',
 '4385',
 '30735',
 '12982',
 '16634',
 '31315',
 '11676',
 '277639',
 '20115',
 '12538',
 '23288',
 '21659',
 '28204',
 '25409',
 '7346',
 '8067',
 '6543',
 '32440',
 '277427',
 '4017',
 '13273',
 '17950',
 '20859',
 '24921',
 '7158',
 '277478',
 '6242',
 '26516',
 '33974',
 '27647',
 '14521',
 '6323',
 '35859',
 '15957',
 '30972',
 '18401',
 '22625',
 '26583',
 '28634',
 '30276',
 '21014',
 '35857',
 '3363',
 '33145',
 '6563',
 '28523',
 '278418',
 '31556',
 '26535',
 '254',
 '7286',

In [325]:
# filters the ratings DataFrame to include only rows with user IDs that are in y—that is, 
# only ratings by users with more than 200 ratings.
ratings = ratings.filter(ratings.user_id.isin(user_id_list))

In [326]:
ratings.show(5)
print(ratings.count(), len(ratings.columns))

+-------+----------+------+
|user_id|      ISBN|rating|
+-------+----------+------+
| 277427|002542730X|    10|
| 277427|0026217457|     0|
| 277427|003008685X|     8|
| 277427|0030615321|     0|
| 277427|0060002050|     0|
+-------+----------+------+
only showing top 5 rows

526356 3


In [327]:
# join the ratings with books
ratings_with_books = ratings.join(books, on="ISBN", how="inner")

In [344]:
ratings_with_books.show(5)

+----------+-------+------+--------------------+----------------+----+--------------------+--------------------+
|      ISBN|user_id|rating|               title|          author|year|           publisher|           image_url|
+----------+-------+------+--------------------+----------------+----+--------------------+--------------------+
|0001048473|  23902|     0|Nothing Can Be Be...|Barns &amp; Budd|1996|            Atlantic|http://images.ama...|
|0001382381|  26583|     0|Huck Scarry's Ste...|     Huck Scarry|1979|HarperCollins Pub...|http://images.ama...|
|0001848445|  11676|     0| THE COAL HOUSE T/PB|   Andrew Taylor|1986|HarperCollins Pub...|http://images.ama...|
|0001848445| 131402|     0| THE COAL HOUSE T/PB|   Andrew Taylor|1986|HarperCollins Pub...|http://images.ama...|
|0001900277|  11676|     0|Glue (First Facts...|   Harriet Hains|1989|HarperCollins Pub...|http://images.ama...|
+----------+-------+------+--------------------+----------------+----+--------------------+-----

In [329]:
print(ratings_with_books.count(), len(ratings_with_books.columns))

487685 8


In [330]:
from pyspark.sql.functions import count

# Group by 'title' and count the number of ratings for each title.
# In Spark, the result is already a DataFrame with columns (no need for a separate reset_index() as in Pandas).
number_rating = ratings_with_books.groupBy("title").agg(count("rating").alias("rating_num"))

In [ ]:
number_rating.show(5)

+--------------------+----------+
|               title|rating_num|
+--------------------+----------+
|Survival Guide to...|         1|
|     In the Clearing|         1|
|Too Many Men : A ...|         4|
|Emily Post's Wedd...|         1|
|Speculations: The...|         1|
|Cooking in a Beds...|         1|
|The Dream of the ...|         1|
|The Wizard's Holi...|         2|
|Madness and Sexua...|         1|
|Fallen Hero/the S...|         1|
|Last Train to Mem...|         2|
|   A Calculated Risk|        12|
|Sugar Busters! Sh...|         1|
|One Last Fling (T...|         1|
|Reilly'S Bride (S...|         1|
|Heart Versus Humb...|         1|
|A Baby By Christm...|         3|
|Bedroom Therapy (...|         4|
|   Let a Soldier Die|         1|
|Poison Penmanship...|         1|
+--------------------+----------+
only showing top 20 rows



In [332]:
print(number_rating.count(), len(number_rating.columns))

160280 2


In [333]:
ratings = ratings_with_books.join(number_rating, on="title", how="inner")

In [ ]:
ratings.show(5)

+--------------------+----------+-------+------+--------------+----+--------------------+--------------------+----------+
|               title|      ISBN|user_id|rating|        author|year|           publisher|           image_url|rating_num|
+--------------------+----------+-------+------+--------------+----+--------------------+--------------------+----------+
| A Light in the S...|0590567330|  35859|     0|   Karen Hesse|1999|Hyperion Books fo...|http://images.ama...|         2|
| A Light in the S...|0590567330|  96448|     9|   Karen Hesse|1999|Hyperion Books fo...|http://images.ama...|         2|
| Q-Space (Star Tr...|0671019155|  12538|     0|      Greg Cox|1998|           Star Trek|http://images.ama...|        12|
| Q-Space (Star Tr...|0671019155|  30276|     0|      Greg Cox|1998|           Star Trek|http://images.ama...|        12|
| Q-Space (Star Tr...|0671019155|  35859|     0|      Greg Cox|1998|           Star Trek|http://images.ama...|        12|
| Q-Space (Star Tr...|06

In [335]:
print(ratings.count(), len(ratings.columns))

487685 9


In [336]:
ratings = ratings.filter(ratings["rating_num"] >= 50)

In [337]:
ratings.show(5)

+--------------------+----------+-------+------+---------+----+---------+--------------------+----------+
|               title|      ISBN|user_id|rating|   author|year|publisher|           image_url|rating_num|
+--------------------+----------+-------+------+---------+----+---------+--------------------+----------+
|A Thin Dark Line ...|0553571885|   6323|     0|TAMI HOAG|1998|   Bantam|http://images.ama...|        61|
|A Thin Dark Line ...|0553571885|   7915|     0|TAMI HOAG|1998|   Bantam|http://images.ama...|        61|
|A Thin Dark Line ...|0553571885|  11601|     0|TAMI HOAG|1998|   Bantam|http://images.ama...|        61|
|A Thin Dark Line ...|0553571885|  28204|     0|TAMI HOAG|1998|   Bantam|http://images.ama...|        61|
|A Thin Dark Line ...|0553571885|  29259|     0|TAMI HOAG|1998|   Bantam|http://images.ama...|        61|
+--------------------+----------+-------+------+---------+----+---------+--------------------+----------+
only showing top 5 rows



In [338]:
print(ratings.count(), len(ratings.columns))

61853 9


In [339]:
ratings = ratings.dropDuplicates(['user_id', 'title']) # drop the duplicates with the same user_id / title

In [340]:
print(ratings.count(), len(ratings.columns))

59850 9


In [341]:
from pyspark.sql.functions import avg

# Pivot the table with user_id as columns, title as rows, and average rating as values
book_pivot = ratings.groupBy("title") \
                         .pivot("user_id") \
                         .agg(avg("rating")) \
                         .fillna(0)  # Optional: Replace nulls with 0